In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Nous allons commencé par l'importation des librairies !!! :) 

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import scipy

%matplotlib inline

Ensuite télécharger le dataset et explorer les données qu'il contient !!! 

In [ ]:
train = pd.read_csv ('../input/train.csv')
test = pd.read_csv ('../input/test.csv')
sample_submission = pd.read_csv ('../input/sample_submission.csv')

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
test.head()

In [ ]:
test.tail()

In [ ]:
train.info()

In [ ]:
test.info()

Information sur le dataFields !!! 

1/ id - a unique identifier for each trip
2/ vendor_id - a code indicating the provider associated with the trip record
3/ pickup_datetime - date and time when the meter was engaged
4/ dropoff_datetime - date and time when the meter was disengaged
5/ passenger_count - the number of passengers in the vehicle (driver entered value)
6/ pickup_longitude - the longitude where the meter was engaged
7/ pickup_latitude - the latitude where the meter was engaged
8/ dropoff_longitude - the longitude where the meter was disengaged
9/ dropoff_latitude - the latitude where the meter was disengaged
10/ store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip
11/ trip_duration - duration of the trip in seconds

Disclaimer: The decision was made to not remove dropoff coordinates from the dataset order to provide an expanded set of variables to use in Kernels.

In [ ]:
train.describe()

On remarque qu'il y a existance d'une echelle ( de valeur comprise entre un x et un y) !!!

In [ ]:
train.select_dtypes

In [ ]:
train.dtypes

In [ ]:
test.dtypes

Le dtype est un object

Nous allons calculer la durée minimale et maximale d'un déplacement  dans la ville de New York !!!



In [ ]:
train.trip_duration.min()

In [ ]:
train.trip_duration.max()

La durée minimale d'un trip à New York est compris entre 1scd et 3526282 scd !!!  

In [ ]:
train["pickup_datetime"].head()

In [ ]:
train["dropoff_datetime"].head()

In [ ]:
train["store_and_fwd_flag"].head()

DATA PROCESSING !!!

1/ Répartition des outliers et description du train !!!

In [ ]:
plt.subplots(figsize=(18,7))
plt.title("outliers repartition")
train.boxplot()

Après avoir vu les Outliers , le "trip-duration" attire le plus notre attention, nous allons nous concentré sur ce point alors !!! et checker ses valeurs qui sont inférieures à 5000 !!! :) 

In [ ]:
train.loc[train.trip_duration<5000,"trip_duration"].hist(bins=200,
                                                        color= "#FF1493"
                                                        )

Les trip_duration font presque tous moins de 1h30, c'est donc la raison pour laquelle nous allons resté focussed sur ces valeurs 

In [ ]:
train['log_trip_duration'] = np.log(train['trip_duration'])
plt.hist(train['log_trip_duration'].values, bins=150,color= "#A569BD" )
plt.xlabel('log(trip_duration)')
plt.ylabel('number of train records')
plt.show()

Nous avons donc réaliser le log_Trip_Duration !!! 

In [ ]:
import matplotlib.pyplot as plt 
from matplotlib import animation
from matplotlib import cm
import base64
import io
longitude = list(train.pickup_longitude) + list(train.dropoff_longitude)
latitude = list(train.pickup_latitude) + list(train.dropoff_latitude)
plt.figure(figsize = (10, 10))
plt.plot(longitude,latitude,'.', alpha = 1, markersize = 25)
plt.show()

2/ Les missing values et les valeurs en doubles !!

In [ ]:
train.isnull().sum()

In [ ]:
train.dropna(inplace=True)
train.isnull().sum()

 on remarque qu'il n'y a pas de missing values on va voir alors combien de valeurs nous allions avoir !!

In [ ]:
train = train[(train.trip_duration < 5000)]
train.info()

Nous sommes donc passés de 1458644 à 1454870 donc moins de 3774 valeurs perdues ( innexploitables ) dans notre data training !!!

3/ Les categorical variables !!!

Nous avons remarqué que le temps est donné en seconds, ce qui nous arrange pas , du coup nous allons le convertir !!!

In [ ]:
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
test['pickup_datetime'] = pd.to_datetime(test['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
train['hour'] = train.loc[:,'pickup_datetime'].dt.hour;
train['week'] = train.loc[:,'pickup_datetime'].dt.week;
train['weekday'] = train.loc[:,'pickup_datetime'].dt.weekday;
train['hour'] = train.loc[:,'pickup_datetime'].dt.hour;
train['month'] = train.loc[:,'pickup_datetime'].dt.month;

In [ ]:
test['hour'] = test.loc[:,'pickup_datetime'].dt.hour;
test['week'] = test.loc[:,'pickup_datetime'].dt.week;
test['weekday'] = test.loc[:,'pickup_datetime'].dt.weekday;
test['hour'] = test.loc[:,'pickup_datetime'].dt.hour;
test['month'] = test.loc[:,'pickup_datetime'].dt.month;

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.shape

In [ ]:
test.shape

Comme remarqué tout en haut loes de l'exploration du dataset, que les variables catégoriques seront : pickup_datetime  et dropoff_datetime , parcontre store_and_fwd_flag  nous ne sera d'aucune importance mm object , d'autant plus N=not a store and forward trip !!! 

In [ ]:
cat_vars = ['store_and_fwd_flag']

In [ ]:
for col in cat_vars:
    train[col] = train[col].astype('category').cat.codes
train.head()

In [ ]:
for col in cat_vars:
    test[col] = test[col].astype('category').cat.codes
test.head()

In [ ]:
train.columns

Feature engeneering

Création des features à partir de notre data existante afin de train notre modele de machine learning.

In [ ]:
selection_train = ["passenger_count","store_and_fwd_flag", "pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude", "week", "weekday", "hour", "month"]
selection_test = ["passenger_count","store_and_fwd_flag", "pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude", "week", "weekday", "hour", "month"]

#selection_train = ["passenger_count","month", "weekday", "hour", "week", "store_and_fwd_flag"]
#selection_test = ["passenger_count", "month", "weekday", "hour", "week", "store_and_fwd_flag"]

y_train = np.log(train["trip_duration"]) # ma target 
X_train = train[selection_train] # ses features
X_test = test[selection_test]


In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
X = np.random.randn(10, 3)
scaler = StandardScaler()
scaler.fit(X)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train.shape, X_test.shape, y_train.shape

In [ ]:
y_train.head(20)

In [ ]:
X_train.head(20)

In [ ]:
X_test.head(20)

Pour réaliser le modeling Nous allons Train avec RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
mrf = RandomForestRegressor(n_estimators=20)
mrf.fit(X_train, y_train)

Passer à la validation 

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
#crossval_scores = cross_val_score(mrf, X_train, y_train, cv=5, scoring='neg_mean_squared_log_error')
#crossval_scores

In [ ]:
#for i in range(len(crossval_scores)):
#    crossval_scores[i] = np.sqrt(abs(crossval_scores[i]))
#crossval_scores

Nous allons réaliser les prédictions!!

In [ ]:
y_test_pred = mrf.predict(X_test)
y_test_pred[:5]

On va Submit !!!

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv')
submission.head()

In [ ]:
submission['trip_duration']= np.exp(y_test_pred)
submission.to_csv('asmasem_submission.csv', index=False)

In [ ]:
submission.head(10)